In [ ]:
#
#   Copyright sk3pta 2024
#   Случайная цитата для этого файла :
#   Каждый должен оставить свой след и чего-то добиться
#

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

from sklearn.metrics import classification_report,confusion_matrix
#https://habr.com/ru/articles/802185/


In [4]:
df = pd.read_csv('diabetes.csv')

# By default, this function will create a grid of Axes such that each numeric variable in data will by shared across the y-axes across a single row and the x-axes across a single column. The diagonal plots are treated differently: a univariate distribution plot is drawn to show the marginal distribution of the data in each column.
  
#sns.pairplot(df, hue="Outcome")


x = df.drop('Outcome',axis = 1)
y = df['Outcome']



In [7]:

#sklearn.model_selection.train_test_split(*arrays, test_size=None, train_size=None, random_state=None, shuffle=True, stratify=None
"""
test_size можно в процентах
random_state  Whenever randomization is part of a Scikit-learn algorithm, a random_state parameter may be provided to control the random number generator used. N

"""

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.30)

In [9]:
svc_model = SVC()
svc_model.fit(x_train, y_train)


SVC()

In [21]:
# теперь проверяем
predict = svc_model.predict(x_test)

#print(y_test)
#print(y)

conf_mx = confusion_matrix(y_test, predict)
print(f"    Positive    Negative    ")
print(f"       {conf_mx[0][0]}         {conf_mx[0][1]}")
print(f"       {conf_mx[1][0]}         {conf_mx[1][1]}")
print()

# true positive = предсказанный и реальный выход совпали
# false positive = предсказанный результат позитивный, но целевой - нет
# и т. д
#   Pos Neg

#   TP  FP
#   FN  TN

# recall = tp / (tp + fn)
# precision = tp / (tp +fp)
# accur     = (tp + tn) / (tp + tn + fp + fn)
# f1        = ( 2 * prec * rec) / (prec + rec)

# macro avg Рассчитывает среднее метрических показателей для каждого класса, взвешивая их одинаково:
# weighted avg Рассчитывает среднее метрических оценок для каждого класса, взвеши­вая их пропорционально представительству в сведениях:

print(classification_report(y_test, predict))

    Positive    Negative    
       139         12
       37         43

              precision    recall  f1-score   support

           0       0.79      0.92      0.85       151
           1       0.78      0.54      0.64        80

    accuracy                           0.79       231
   macro avg       0.79      0.73      0.74       231
weighted avg       0.79      0.79      0.78       231

